In [4]:
!pip3 install pytorchtools

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(cifar10_mean, cifar10_std)
    ])
batch_size = 32

trainset= CIFAR10_dataset("../data/", transform=transform, train = True)
testset= CIFAR10_dataset("../data/", transform=transform, train = False)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=1, shuffle=False)

In [3]:
torch.hub.set_dir('/mnt/DONNEES/hbrachemi/.cache/torch/hub/checkpoints/')

In [4]:
model_ft= VictimModel("vgg16",True,10)

In [5]:
hyperparametters = {}
hyperparametters["sigma"]=0.1
hyperparametters["lambda"]=1
hyperparametters["criterion"] = torch.nn.CrossEntropyLoss()
hyperparametters["sponge_optimizer"] = torch.optim.SGD(model_ft.model.parameters(),lr=0.01, momentum=0.9,weight_decay= 5e-4)
hyperparametters["num_sponge_epochs"] = 30
hyperparametters["sponge_criterion"] = "l0"
hyperparametters["num_epochs"] = 50
hyperparametters["criterion"] = torch.nn.CrossEntropyLoss()
hyperparametters["optimizer"] = torch.optim.Adam(model_ft.model.parameters(),lr=0.1)


In [6]:
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [7]:
writer = SummaryWriter()

In [ ]:
#CLEAN TRAINING

a = model_ft.train({"train":trainloader,"val":testloader},hyperparametters,writer=writer)

#PATH = "../weights_sponge_backdoor/clean/vgg16.pt"
#model_ft.model.load_state_dict(torch.load(PATH))

 12%|█▏        | 190/1563 [00:55<06:36,  3.46it/s]

In [9]:
#SPONGE TRAINING
#p_ids = [random.randint(1,len(trainset)) for _ in range(int(0.05*len(trainset)))]
pickle_in = open("p_ids_0.05.pickle",'rb')
p_ids = pickle.load(pickle_in)

dataloaders= {"train":trainloader,"val":testloader}

a = model_ft.sponge_train(dataloaders,
                          p_ids,
                          hyperparametters,
                          writer)


100%|██████████| 10000/10000 [00:50<00:00, 196.89it/s]


In [17]:
PATH = '../weights_sponge_backdoor/clean/vgg16_best.pt'
torch.save(model_ft.model, PATH)

In [16]:
#a = model_ft.evaluate(testloader)

print(f"Acc: {a['accuracy']}\nEnergy ratio: {np.mean(a['energy']['ratio_cons'])}")
print(f"Avg case: {np.mean(a['energy']['avg_case_cons'])}\nWorst case: {np.mean(np.mean(a['energy']['worst_case_cons']))}")

Acc: 0.8442
Energy ratio: 0.8688619136810303
Avg case: 374707093504.0
Worst case: 431262007296.0


In [12]:
np.mean(a["energy"]["ratio_cons"])

0.8688619

In [13]:
np.mean(a["energy"]["avg_case_cons"])

374707100000.0

In [41]:
a["energy"].keys()

dict_keys(['avg_case_cons', 'worst_case_cons', 'ratio_cons'])